In [4]:
import sys
import random
import os
from pathlib import Path
import shutil
import json

import argparse
import tqdm
import spacy
from spacy.gold import minibatch
from spacy.language import Language
from spacy import util

In [5]:
from scispacy.data_util import read_full_med_mentions, read_ner_from_tsv
from scispacy.util import WhitespaceTokenizer
from scispacy.per_class_scorer import PerClassScorer
from scispacy.train_utils import evaluate_ner

In [3]:
def train_ner(output_dir: str,
              train_data_path: str,
              dev_data_path: str,
              test_data_path: str,
              run_test: bool = None,
              model: str = None,
              n_iter: int = 10,
              meta_overrides: str = None):

    util.fix_random_seed(util.env_opt("seed", 0))
    train_data = read_ner_from_tsv(train_data_path)
    dev_data = read_ner_from_tsv(dev_data_path)
    test_data = read_ner_from_tsv(test_data_path)
    os.makedirs(output_dir, exist_ok=True)
    if run_test:
        nlp = spacy.load(model)
        print("Loaded model '%s'" % model)
        evaluate_ner(nlp, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
        evaluate_ner(nlp, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))
    else:
        train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides)

In [4]:
def train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides):
    """Load the model, set up the pipeline and train the entity recognizer."""
    
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    if meta_overrides is not None:
        metadata = json.load(open(meta_overrides))
        nlp.meta.update(metadata)

    original_tokenizer = nlp.tokenizer

    nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names and "parser" in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, after="parser")
    elif 'ner' not in nlp.pipe_names and "tagger" in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, after="tagger")
    elif 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    dropout_rates = util.decaying(util.env_opt('dropout_from', 0.2),
                                  util.env_opt('dropout_to', 0.2),
                                  util.env_opt('dropout_decay', 0.005))
    batch_sizes = util.compounding(util.env_opt('batch_from', 1),
                                   util.env_opt('batch_to', 128),
                                   util.env_opt('batch_compound', 1.001))

    optimizer = nlp.begin_training()
    best_epoch = 0
    best_f1 = 0
    for i in range(n_iter):
        print(str(i)+'--'+str(n_iter))
        random.shuffle(train_data)
        count = 0
        losses = {}
        total = len(train_data)

        with nlp.disable_pipes(*other_pipes):  # only train NER
            with tqdm.tqdm(total=total, leave=True) as pbar:
                for batch in minibatch(train_data, size=batch_sizes):
                    docs, golds = zip(*batch)
                    nlp.update(docs, golds, sgd=optimizer,
                               losses=losses, drop=next(dropout_rates))
                    pbar.update(len(batch))
                    if count % 100 == 0 and count > 0:
                        print('sum loss: %s' % losses['ner'])
                    count += 1

        # save model to output directory
        output_dir_path = Path(output_dir + "/" + str(i))
        if not output_dir_path.exists():
            output_dir_path.mkdir()

        with nlp.use_params(optimizer.averages):
            nlp.tokenizer = original_tokenizer
            nlp.to_disk(output_dir_path)
            print("Saved model to", output_dir_path)

        # test the saved model
        print("Loading from", output_dir_path)
        nlp2 = util.load_model_from_path(output_dir_path)
        nlp2.tokenizer = WhitespaceTokenizer(nlp.vocab)

        metrics = evaluate_ner(nlp2, dev_data)
        if metrics["f1-measure-overall"] > best_f1:
            best_f1 = metrics["f1-measure-overall"]
            best_epoch = i
    # save model to output directory
    best_model_path = Path(output_dir + "/" + "best")
    print(f"Best Epoch: {best_epoch} of {n_iter}")
    if os.path.exists(best_model_path):
        shutil.rmtree(best_model_path)
    shutil.copytree(os.path.join(output_dir, str(best_epoch)),
                    best_model_path)

    # test the saved model
    print("Loading from", best_model_path)
    nlp2 = util.load_model_from_path(best_model_path)
    nlp2.tokenizer = WhitespaceTokenizer(nlp.vocab)

    evaluate_ner(nlp2, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
    evaluate_ner(nlp2, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))

In [5]:
model_output_dir = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pubmed-pmc-sm/'
train_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/EBI_standard-IOB/train.csv'
dev_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/EBI_standard-IOB/dev.csv'
test_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/EBI_standard-IOB/test.csv'
run_test = False
model_path = '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/pubmed-pmc-sm' # None #'en_core_sci_md'
iterations = 10
meta_overrides = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/scispacy/data/EPMC_ner.json'

In [6]:
train_ner(model_output_dir,
              train_data_path,
              dev_data_path,
              test_data_path,
              run_test,
              model_path,
              iterations,
              meta_overrides)

Loaded model '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/pubmed-pmc-sm'


  0%|          | 1/79401 [00:00<2:38:10,  8.37it/s]

0--10


  0%|          | 104/79401 [00:03<36:27, 36.25it/s] 

sum loss: 231.20640152121283


...

precision-GP: 0.8052016985138004
recall-GP: 0.7447226313205695
f1-measure-GP: 0.7737821984187208
precision-untyped: 0.8074424099232133
recall-untyped: 0.7494517543859649
f1-measure-untyped: 0.7773670742109254
precision-DS: 0.741119807344973
recall-DS: 0.6857938718662953
f1-measure-DS: 0.7123842592592093
precision-OG: 0.8407732864674868
recall-OG: 0.7873601053324556
f1-measure-OG: 0.8131905490395558
precision-overall: 0.8046859618035046
recall-overall: 0.7468932748538012
f1-measure-overall: 0.7747132973177399


 96%|█████████▋| 17049/17680 [00:50<00:01, 361.00it/s]

precision-GP: 0.8087662887121341
recall-GP: 0.7517697026899481
f1-measure-GP: 0.779227131909293
precision-untyped: 0.812580823942361
recall-untyped: 0.7533613085552796
f1-measure-untyped: 0.7818513087143438
precision-DS: 0.7599403578528827
recall-DS: 0.6890491212257773
f1-measure-DS: 0.7227605766957668
precision-OG: 0.8430292598967297
recall-OG: 0.7897452434698484
f1-measure-OG: 0.8155178155177655
precision-overall: 0.808886015148716
recall-overall: 0.7499357711741029
f1-measure-overall: 0.7782962271696608


100%|██████████| 17680/17680 [00:51<00:00, 342.77it/s]


precision-untyped: 		 0.8126202033153219
recall-untyped: 		 0.7529062367416207
f1-measure-untyped: 		 0.7816243833685195
precision-overall: 		 0.8089568641817017
recall-overall: 		 0.7495120916419177
f1-measure-overall: 		 0.7781007751937487
